# 01 — Prepare PV & Load (authoritative clean)
This notebook:
1) Robustly loads `data/interim/pv_5homes_10min.csv` and `data/interim/load_5homes_10min.csv`
2) Detects timestamp column and numeric columns
3) Aligns on **time only**
4) Normalizes columns to `h1..hN`
5) Detects/repairs unit mismatch (PV in kWh/10min vs Load in kW → multiply PV by 6)
6) Writes cleaned artifacts to `data/processed/` and a quick meta report to `reports/`


In [29]:
def recommend_for(home: str, ts: pd.Timestamp,
                  surplus_df: pd.DataFrame,
                  can_sell_df: pd.DataFrame | None = None) -> dict:
    """
    At time ts:
      - Determine selected home's status (HIGH/LOW/NEUTRAL) from surplus sign.
      - If LOW: recommend the other home with the LARGEST surplus (seller).
      - If HIGH: recommend the other home with the LARGEST deficit (buyer).
      - Return a market snapshot listing all sellers and buyers at ts.
    """
    # Current home's surplus
    s_home = float(surplus_df.at[ts, home])

    # Whole-row view at this slot
    row_all = surplus_df.loc[ts, :].astype(float)

    # Sellers = positive surplus; honor can_sell if provided
    sellers_mask = row_all > 0
    if can_sell_df is not None:
        sellers_mask = sellers_mask & can_sell_df.loc[ts, :].astype(bool)
    sellers = row_all[sellers_mask].sort_values(ascending=False)  # biggest surplus first

    # Buyers = negative surplus (deficit)
    buyers = row_all[row_all < 0].sort_values(ascending=True)     # most negative (largest deficit) first

    # Default recommendations
    best_seller = None
    best_seller_surplus = None
    best_buyer = None
    best_buyer_deficit = None

    # For a LOW home, pick best seller (exclude self)
    if s_home < 0:
        cand = sellers.drop(labels=[home], errors="ignore")
        if not cand.empty:
            best_seller = cand.index[0]
            best_seller_surplus = float(cand.iloc[0])

    # For a HIGH home, pick best buyer (exclude self)
    if s_home > 0:
        cand_b = buyers.drop(labels=[home], errors="ignore")
        if not cand_b.empty:
            best_buyer = cand_b.index[0]
            best_buyer_deficit = float(cand_b.iloc[0])  # negative value

    status = "HIGH" if s_home > 0 else ("LOW" if s_home < 0 else "NEUTRAL")

    result = {
        "home": home,
        "timestamp": ts,
        "surplus_home_kWh": s_home,
        "status": status,
        "advice": None,
        "reason": None,
        "best_seller": best_seller,
        "best_seller_surplus_kWh": best_seller_surplus,
        "best_buyer": best_buyer,
        "best_buyer_deficit_kWh": best_buyer_deficit,  # negative number
        "snapshot": {
            "sellers": sellers,   # pd.Series, positive kWh
            "buyers": buyers      # pd.Series, negative kWh
        }
    }

    # Advice text for both sides
    if status == "LOW":
        if best_seller is not None:
            result["advice"] = f"{home} is low ({s_home:.2f} kWh deficit). Buy from {best_seller}."
            result["reason"] = f"{best_seller} has the most excess at this slot: {best_seller_surplus:.2f} kWh."
        else:
            result["advice"] = f"{home} is low ({s_home:.2f} kWh deficit). No internal sellers now."
            result["reason"] = "No home with positive surplus at this slot; use battery or grid."
    elif status == "HIGH":
        if best_buyer is not None:
            # best_buyer_deficit is negative → show absolute value in the reason
            result["advice"] = f"{home} has surplus ({s_home:.2f} kWh). Sell to {best_buyer}."
            result["reason"] = f"{best_buyer} needs the most energy now: {abs(best_buyer_deficit):.2f} kWh deficit."
        else:
            result["advice"] = f"{home} has surplus ({s_home:.2f} kWh). No buyers right now."
            result["reason"] = "No home with a deficit at this slot; consider charging or storing."
    else:
        result["advice"] = f"{home} is balanced (≈0 kWh). Hold / recheck next slot."
        result["reason"] = "Net surplus is ~0 kWh."

    return result


In [38]:
# === Cell B: Interactive run (REPLACE this whole cell) ===

print("What home are you? (e.g., h1..h5)")
home = parse_home(input())

day0 = get_valid_day_by_number()      # you enter ONLY the day number (e.g., 23)
offset = parse_time_10min()

ts_raw = pd.Timestamp(day0 + offset)
ts = nearest_slot(ts_raw, surplus.index)

res = recommend_for(home, ts, surplus, can_sell)

s = float(res["surplus_home_kWh"])
status = res["status"]

print("\n--- Decision ---")
print(f"Slot selected: {ts} (nearest to your input {ts_raw})")
print(f"Home: {res['home']}  |  Status: {status}")

# Always show signed net
print(f"Your net: {s:+.2f} kWh")

# Counterparty suggestion depending on HIGH/LOW
if status == "LOW":
    if res["best_seller"] is not None:
        print(f"Recommended seller: {res['best_seller']}  (+{res['best_seller_surplus_kWh']:.2f} kWh)")
    else:
        print("Recommended seller: —")
elif status == "HIGH":
    if res["best_buyer"] is not None:
        need = abs(res["best_buyer_deficit_kWh"]) if res["best_buyer_deficit_kWh"] is not None else 0.0
        print(f"Recommended buyer:  {res['best_buyer']}  (needs {need:.2f} kWh)")
    else:
        print("Recommended buyer:  —")
else:
    print("Recommended counterparty: —")

print(f"\nAdvice: {res['advice']}")
print(f"Reason: {res['reason']}")

# Market snapshot
sellers = res["snapshot"]["sellers"]
buyers  = res["snapshot"]["buyers"]

print("\n--- Sellers at this slot (descending by surplus) ---")
if sellers.empty:
    print("None")
else:
    for h, v in sellers.items():
        print(f"{h}: {v:+.2f} kWh")  # always show sign

print("\n--- Buyers at this slot (descending by deficit) ---")
if buyers.empty:
    print("None")
else:
    for h, v in buyers.items():
        print(f"{h}: {v:+.2f} kWh")  # negative numbers display with a minus


What home are you? (e.g., h1..h5)
Available days (day numbers only): 23, 24, 25, 26

--- Decision ---
Slot selected: 2018-08-24 19:10:00+00:00 (nearest to your input 2018-08-24 19:10:00)
Home: h2  |  Status: LOW
Your net: -3.11 kWh
Recommended seller: h5  (+2.85 kWh)

Advice: h2 is low (-3.11 kWh deficit). Buy from h5.
Reason: h5 has the most excess at this slot: 2.85 kWh.

--- Sellers at this slot (descending by surplus) ---
h5: +2.85 kWh
h4: +2.08 kWh
h3: +0.62 kWh

--- Buyers at this slot (descending by deficit) ---
h2: -3.11 kWh
h1: -1.71 kWh
